In [1]:
import os
import random
import shutil
import pandas as pd
from PIL import Image, UnidentifiedImageError

# Function to pick random images from each disease folder and copy them into a new folder
def pick_random_images(source_dir, num_images, label_prefix, output_dir, ground_truth_csv):
    random_images = []
    for label in os.listdir(source_dir):
        label_dir = os.path.join(source_dir, label)
        if os.path.isdir(label_dir):
            images = os.listdir(label_dir)
            selected_images = random.sample(images, min(num_images, len(images)))  # Pick 50 or as many available images
            for image in selected_images:
                image_path = os.path.join(label_dir, image)
                
                # Check if the image is valid and readable
                try:
                    with Image.open(image_path) as img:
                        img.verify()  # Verify the image is not corrupted
                    
                    # Create new image name, e.g., "f_1_453_36" for Fibrosis with score 1
                    new_image_name = f"{label_prefix}_{label}_{image.split('.')[0]}"
                    new_image_path = os.path.join(output_dir, new_image_name + ".jpg")
                    
                    # Copy image to the new folder and store the ground truth in the CSV
                    shutil.copy(image_path, new_image_path)
                    random_images.append([new_image_name, label_prefix, int(label)])  # Append the ground truth label (as integer)
                
                except (UnidentifiedImageError, IOError):
                    # Skip unreadable images like thumbnails or corrupted files
                    print(f"Skipped unreadable image: {image_path}")
    
    # Save the ground truth data in a CSV
    df = pd.DataFrame(random_images, columns=["Image Name", "Ground Truth Prefix", "Ground Truth Score"])
    df.to_csv(ground_truth_csv, mode='a', header=not os.path.exists(ground_truth_csv), index=False)

# Base directory for all four disease datasets
base_dirs = {
    "Fibrosis": "D:/DATASET/CNN/fibrosis/test",
    "Inflammation": "D:/DATASET/CNN/inflammation/test",
    "Ballooning": "D:/DATASET/CNN/ballooning/test",
    "Steatosis": "D:/DATASET/CNN/steatosis/test"
}

# Output directories for saving random images
output_base_dir = "D:/PATENT/random_test_images"
os.makedirs(output_base_dir, exist_ok=True)
random_image_dirs = {
    "Fibrosis": os.path.join(output_base_dir, "Fibrosis"),
    "Inflammation": os.path.join(output_base_dir, "Inflammation"),
    "Ballooning": os.path.join(output_base_dir, "Ballooning"),
    "Steatosis": os.path.join(output_base_dir, "Steatosis")
}

# Create directories for saving random images
for disease, dir_path in random_image_dirs.items():
    os.makedirs(dir_path, exist_ok=True)

# Ground truth CSV file path
ground_truth_csv = "D:/PATENT/afterGEOMbal.csv"

# Pick 50 random images for each disease and save them with ground truth in CSV
pick_random_images(base_dirs["Fibrosis"], 50, "f", random_image_dirs["Fibrosis"], ground_truth_csv)
pick_random_images(base_dirs["Inflammation"], 50, "i", random_image_dirs["Inflammation"], ground_truth_csv)
pick_random_images(base_dirs["Ballooning"], 50, "b", random_image_dirs["Ballooning"], ground_truth_csv)
pick_random_images(base_dirs["Steatosis"], 50, "s", random_image_dirs["Steatosis"], ground_truth_csv)

print("Random images copied, unreadable images skipped, and ground truth saved in CSV.")

Skipped unreadable image: D:/DATASET/CNN/inflammation/test\1\Thumbs.db
Skipped unreadable image: D:/DATASET/CNN/steatosis/test\3\Thumbs.db
Random images copied, unreadable images skipped, and ground truth saved in CSV.


In [2]:
#only preprocessing for fibrosis rest same as before

In [3]:
# NEW LOGIC OF CALCULATING METRICES

In [2]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import load_model

# Load the models for each disease
fibrosis_model = load_model("D:/PATENT/Model_testing/fibrosis_modified_model25.h5")
inflammation_model = load_model("D:/PATENT/Model_testing/inflam_balanced_modified25.h5")
ballooning_model = load_model("D:/PATENT/Model_testing/SMOTE_ballooning10.h5")
steatosis_model = load_model("D:/PATENT/Model_testing/steatosis_modified15.h5")

# Function to preprocess images for fibrosis
def preprocess_image_fibrosis(image_path):
    img = load_img(image_path, target_size=(299, 299))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    return img_array

# Function to predict image scores based on model type
def predict_image_scores(image_name, image_path):
    if image_name.startswith('f'):
        img_array = preprocess_image_fibrosis(image_path)
        model = fibrosis_model
    else:
        img = load_img(image_path, target_size=(299, 299))
        img_array = img_to_array(img) / 255.0
        img_array = np.expand_dims(img_array, axis=0)

        if image_name.startswith('i'):
            model = inflammation_model
        elif image_name.startswith('b'):
            model = ballooning_model
        elif image_name.startswith('s'):
            model = steatosis_model
        else:
            raise ValueError(f"Unknown disease prefix for {image_name}")

    predictions = model.predict(img_array)
    return np.argmax(predictions) if predictions.shape[-1] > 1 else int(predictions[0][0] > 0.5)

# Calculate metrics based on the new logic
def calculate_metrics(predicted_score, ground_truth_score):
    if ground_truth_score == predicted_score :
        return "TP", 1, 0, 0, 0
    elif ground_truth_score == 0 and predicted_score != 0:
        return "FP", 0, 1, 0, 0
    elif ground_truth_score != 0 and predicted_score == 0:
        return "FN", 0, 0, 0, 1
    elif ground_truth_score != predicted_score:
        return "TN", 0, 0, 1, 0
    else:
        return "", 0, 0, 0, 0

# Predict and save results along with individual metrics
def predict_and_save_results(csv_path, random_image_dirs):
    df = pd.read_csv(csv_path, dtype={'Ground Truth Prefix': str, 'Ground Truth Score': int})
    df['Predicted Prefix'] = ""
    df['Predicted Score'] = np.nan
    df['Performance'] = ""
    df['TP'], df['FP'], df['TN'], df['FN'] = 0, 0, 0, 0

    # Track metrics for each model
    metrics = {
        'f': {'TP': 0, 'FP': 0, 'TN': 0, 'FN': 0},
        'i': {'TP': 0, 'FP': 0, 'TN': 0, 'FN': 0},
        'b': {'TP': 0, 'FP': 0, 'TN': 0, 'FN': 0},
        's': {'TP': 0, 'FP': 0, 'TN': 0, 'FN': 0}
    }

    for idx, row in df.iterrows():
        image_name = row['Image Name']
        ground_truth_prefix = row['Ground Truth Prefix']
        ground_truth_score = row['Ground Truth Score']

        if ground_truth_prefix == 'f':
            image_dir = random_image_dirs["Fibrosis"]
        elif ground_truth_prefix == 'i':
            image_dir = random_image_dirs["Inflammation"]
        elif ground_truth_prefix == 'b':
            image_dir = random_image_dirs["Ballooning"]
        elif ground_truth_prefix == 's':
            image_dir = random_image_dirs["Steatosis"]
        else:
            continue

        image_path = os.path.join(image_dir, image_name + ".jpg")
        predicted_score = predict_image_scores(image_name, image_path)
        performance, tp, fp, tn, fn = calculate_metrics(predicted_score, ground_truth_score)

        df.at[idx, 'Predicted Prefix'] = ground_truth_prefix
        df.at[idx, 'Predicted Score'] = predicted_score
        df.at[idx, 'Performance'] = performance
        df.at[idx, 'TP'] = tp
        df.at[idx, 'FP'] = fp
        df.at[idx, 'TN'] = tn
        df.at[idx, 'FN'] = fn

        metrics[ground_truth_prefix]['TP'] += tp
        metrics[ground_truth_prefix]['FP'] += fp
        metrics[ground_truth_prefix]['TN'] += tn
        metrics[ground_truth_prefix]['FN'] += fn

    # Calculate and print metrics for each model
    for prefix, data in metrics.items():
        tp, fp, tn, fn = data['TP'], data['FP'], data['TN'], data['FN']
        total_predictions = tp + fp + tn + fn

        precision = tp / (tp + fp) if (tp + fp) != 0 else 0
        recall = tp / (tp + fn) if (tp + fn) != 0 else 0
        f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) != 0 else 0
        iou = tp / (tp + fp + fn) if (tp + fp + fn) != 0 else 0
        accuracy = (tp + tn) / total_predictions if total_predictions != 0 else 0

        print(f"\nMetrics for {prefix.upper()} model:")
        print(f"  Accuracy: {accuracy:.2f}")
        print(f"  Precision: {precision:.2f}")
        print(f"  Recall: {recall:.2f}")
        print(f"  F1-Score: {f1_score:.2f}")
        print(f"  IoU: {iou:.2f}")

    df.to_csv(csv_path, index=False)
    print("\nPredictions completed and saved with performance metrics in CSV.")

# Base path to random images for each disease
random_image_dirs = {
    "Fibrosis": "D:/PATENT/random_test_images/Fibrosis",
    "Inflammation": "D:/PATENT/random_test_images/Inflammation",
    "Ballooning": "D:/PATENT/random_test_images/Ballooning",
    "Steatosis": "D:/PATENT/random_test_images/Steatosis"
}

# Path to the ground truth CSV file
csv_path = "D:/PATENT/afterSMOTEbal.csv"

# Predict and save results for all diseases
predict_and_save_results(csv_path, random_image_dirs)


ValueError: Unknown loss function: focal_loss_fixed. Please ensure this object is passed to the `custom_objects` argument. See https://www.tensorflow.org/guide/keras/save_and_serialize#registering_the_custom_object for details.

In [ ]:
# new csv file containing the metrices values except accuracy

In [ ]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import load_model

# Load the models for each disease
fibrosis_model = load_model("D:/PATENT/Model_testing/fibrosis_modified_model25.h5")
inflammation_model = load_model("D:/PATENT/Model_testing/inflam_balanced_modified25.h5")
ballooning_model = load_model("D:/PATENT/Model_testing/model_modified_ballooning25.h5")
steatosis_model = load_model("D:/PATENT/Model_testing/steatosis_modified15.h5")

# Function to preprocess images for fibrosis
def preprocess_image_fibrosis(image_path):
    img = load_img(image_path, target_size=(299, 299))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    return img_array

# Function to predict image scores based on model type
def predict_image_scores(image_name, image_path):
    if image_name.startswith('f'):
        img_array = preprocess_image_fibrosis(image_path)
        model = fibrosis_model
    else:
        img = load_img(image_path, target_size=(299, 299))
        img_array = img_to_array(img) / 255.0
        img_array = np.expand_dims(img_array, axis=0)

        if image_name.startswith('i'):
            model = inflammation_model
        elif image_name.startswith('b'):
            model = ballooning_model
        elif image_name.startswith('s'):
            model = steatosis_model
        else:
            raise ValueError(f"Unknown disease prefix for {image_name}")

    predictions = model.predict(img_array)
    return np.argmax(predictions) if predictions.shape[-1] > 1 else int(predictions[0][0] > 0.5)

# Calculate metrics based on the new logic
def calculate_metrics(predicted_score, ground_truth_score):
    if ground_truth_score == predicted_score :
        return "TP", 1, 0, 0, 0
    elif ground_truth_score == 0 and predicted_score != 0:
        return "FP", 0, 1, 0, 0
    elif ground_truth_score != 0 and predicted_score == 0:
        return "FN", 0, 0, 0, 1
    elif ground_truth_score != predicted_score:
        return "TN", 0, 0, 1, 0
    else:
        return "", 0, 0, 0, 0

# Predict and save results along with individual metrics
def predict_and_save_results(csv_path, random_image_dirs):
    df = pd.read_csv(csv_path, dtype={'Ground Truth Prefix': str, 'Ground Truth Score': int})
    df['Predicted Prefix'] = ""
    df['Predicted Score'] = np.nan
    df['Performance'] = ""
    df['TP'], df['FP'], df['TN'], df['FN'] = 0, 0, 0, 0

    # Add new columns for precision, recall, F1-score, IoU
    df['Precision'], df['Recall'], df['F1_Score'], df['IoU'] = np.nan, np.nan, np.nan, np.nan

    metrics = {
        'f': {'TP': 0, 'FP': 0, 'TN': 0, 'FN': 0},
        'i': {'TP': 0, 'FP': 0, 'TN': 0, 'FN': 0},
        'b': {'TP': 0, 'FP': 0, 'TN': 0, 'FN': 0},
        's': {'TP': 0, 'FP': 0, 'TN': 0, 'FN': 0}
    }

    for idx, row in df.iterrows():
        image_name = row['Image Name']
        ground_truth_prefix = row['Ground Truth Prefix']
        ground_truth_score = row['Ground Truth Score']

        if ground_truth_prefix == 'f':
            image_dir = random_image_dirs["Fibrosis"]
        elif ground_truth_prefix == 'i':
            image_dir = random_image_dirs["Inflammation"]
        elif ground_truth_prefix == 'b':
            image_dir = random_image_dirs["Ballooning"]
        elif ground_truth_prefix == 's':
            image_dir = random_image_dirs["Steatosis"]
        else:
            continue

        image_path = os.path.join(image_dir, image_name + ".jpg")
        predicted_score = predict_image_scores(image_name, image_path)
        performance, tp, fp, tn, fn = calculate_metrics(predicted_score, ground_truth_score)

        df.at[idx, 'Predicted Prefix'] = ground_truth_prefix
        df.at[idx, 'Predicted Score'] = predicted_score
        df.at[idx, 'Performance'] = performance
        df.at[idx, 'TP'] = tp
        df.at[idx, 'FP'] = fp
        df.at[idx, 'TN'] = tn
        df.at[idx, 'FN'] = fn

        metrics[ground_truth_prefix]['TP'] += tp
        metrics[ground_truth_prefix]['FP'] += fp
        metrics[ground_truth_prefix]['TN'] += tn
        metrics[ground_truth_prefix]['FN'] += fn

        # Calculate individual row metrics
        precision = tp / (tp + fp) if (tp + fp) != 0 else 0
        recall = tp / (tp + fn) if (tp + fn) != 0 else 0
        f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) != 0 else 0
        iou = tp / (tp + fp + fn) if (tp + fp + fn) != 0 else 0

        # Store metrics in the dataframe
        df.at[idx, 'Precision'] = precision
        df.at[idx, 'Recall'] = recall
        df.at[idx, 'F1_Score'] = f1_score
        df.at[idx, 'IoU'] = iou

    # Save the updated CSV file
    df.to_csv(csv_path, index=False)
    print("\nPredictions completed and saved with performance metrics in CSV.")

# Base path to random images for each disease
random_image_dirs = {
    "Fibrosis": "D:/PATENT/random_test_images/Fibrosis",
    "Inflammation": "D:/PATENT/random_test_images/Inflammation",
    "Ballooning": "D:/PATENT/random_test_images/Ballooning",
    "Steatosis": "D:/PATENT/random_test_images/Steatosis"
}

# Path to the ground truth CSV file
csv_path = "D:/PATENT/ground_truth_Scores_balanced.csv"

# Predict and save results for all diseases
predict_and_save_results(csv_path, random_image_dirs)


In [ ]:
# DO NOT EXECUTE (for referenece)

In [ ]:
# Base directory for all four disease datasets
base_dirs = {
    "Fibrosis": "D:/DATASET/CNN/fibrosis/test",
    "Inflammation": "D:/DATASET/CNN/inflammation/test",
    "Ballooning": "D:/DATASET/CNN/ballooning/test",
    "Steatosis": "D:/DATASET/CNN/steatosis/test"
}

# Output directories for saving random images
output_base_dir = "D:/PATENT/random_test_images"

# Load the models for each disease
fibrosis_model = load_model("D:/PATENT/Model_testing/fibrosis_modified_model25.h5")
inflammation_model = load_model("D:/PATENT/Model_testing/inflam_modified25.h5")
ballooning_model = load_model("D:/PATENT/Model_testing/model_modified_ballooning25.h5")
steatosis_model = load_model("D:/PATENT/Model_testing/steatosis_modified15.h5")


# Base path to random images for each disease
random_image_dirs = {
    "Fibrosis": "D:/PATENT/random_test_images/Fibrosis",
    "Inflammation": "D:/PATENT/random_test_images/Inflammation",
    "Ballooning": "D:/PATENT/random_test_images/Ballooning",
    "Steatosis": "D:/PATENT/random_test_images/Steatosis"
}

# Path to the ground truth CSV file
csv_path = "D:/PATENT/ground_truth_Scores.csv"


In [ ]:
# STARTING NEW TO INCREASE ACCURACY OF BALLOONING AND INFLAMMATION MODELS.

In [ ]:
import os
import random
import shutil
import pandas as pd
from PIL import Image, UnidentifiedImageError
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import load_model

# Load models with error handling
def load_models():
    try:
        fibrosis_model = load_model("D:/PATENT/Model_testing/fibrosis_modified_model25.h5")
        inflammation_model = load_model("D:/PATENT/Model_testing/inflam_balanced_modified25.h5")
        ballooning_model = load_model("D:/PATENT/Model_testing/model_modified_ballooning25.h5")
        steatosis_model = load_model("D:/PATENT/Model_testing/steatosis_modified15.h5")
        return fibrosis_model, inflammation_model, ballooning_model, steatosis_model
    except OSError as e:
        print(f"Error loading models: {e}")
        raise

fibrosis_model, inflammation_model, ballooning_model, steatosis_model = load_models()

# Preprocess image function
def preprocess_image(image_path, target_size=(299, 299), resnet_preprocess=True):
    try:
        img = load_img(image_path, target_size=target_size)
        img_array = img_to_array(img)
        if resnet_preprocess:
            img_array = preprocess_input(img_array)
        img_array = np.expand_dims(img_array, axis=0)
        return img_array
    except (UnidentifiedImageError, IOError):
        print(f"Failed to process image: {image_path}")
        return None

# Predict image scores based on disease prefix
def predict_image_scores(image_name, image_path):
    img_array = preprocess_image(image_path)
    if img_array is None:
        return None  # Skip if the image couldn't be processed

    if image_name.startswith('f'):
        predictions = fibrosis_model.predict(img_array)
    elif image_name.startswith('i'):
        predictions = inflammation_model.predict(img_array)
    elif image_name.startswith('b'):
        predictions = ballooning_model.predict(img_array)
    elif image_name.startswith('s'):
        predictions = steatosis_model.predict(img_array)
    else:
        raise ValueError(f"Unknown disease prefix for {image_name}")

    return np.argmax(predictions) if predictions.shape[-1] > 1 else int(predictions[0][0] > 0.5)

# Calculate performance metrics
def calculate_metrics(tp, fp, tn, fn):
    precision = tp / (tp + fp) if (tp + fp) != 0 else 0
    recall = tp / (tp + fn) if (tp + fn) != 0 else 0
    f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) != 0 else 0
    iou = tp / (tp + fp + fn) if (tp + fp + fn) != 0 else 0
    accuracy = (tp + tn) / (tp + fp + tn + fn) if (tp + fp + tn + fn) != 0 else 0
    return precision, recall, f1_score, iou, accuracy

# Save predictions and metrics
def save_predictions_and_metrics(df, metrics, csv_path):
    for prefix, data in metrics.items():
        tp, fp, tn, fn = data['TP'], data['FP'], data['TN'], data['FN']
        precision, recall, f1_score, iou, accuracy = calculate_metrics(tp, fp, tn, fn)

        print(f"\nMetrics for {prefix.upper()} model:")
        print(f"  Accuracy: {accuracy:.2f}")
        print(f"  Precision: {precision:.2f}")
        print(f"  Recall: {recall:.2f}")
        print(f"  F1-Score: {f1_score:.2f}")
        print(f"  IoU: {iou:.2f}")

    df.to_csv(csv_path, mode='a', index=False, header=not os.path.exists(csv_path))
    print("Predictions and metrics saved successfully.")

# Predict and evaluate images
def predict_and_evaluate(csv_path, image_dirs):
    df = pd.read_csv(csv_path)
    metrics = {k: {'TP': 0, 'FP': 0, 'TN': 0, 'FN': 0} for k in ['f', 'i', 'b', 's']}

    for _, row in df.iterrows():
        image_name, gt_prefix, gt_score = row['Image Name'], row['Ground Truth Prefix'], row['Ground Truth Score']
        image_path = os.path.join(image_dirs[gt_prefix], image_name + ".jpg")

        predicted_score = predict_image_scores(image_name, image_path)
        if predicted_score is None:
            continue

        tp, fp, tn, fn = 0, 0, 0, 0
        if predicted_score == gt_score:
            tp = 1
        elif gt_score == 0 and predicted_score != 0:
            fp = 1
        elif gt_score != 0 and predicted_score == 0:
            fn = 1
        else:
            tn = 1

        metrics[gt_prefix]['TP'] += tp
        metrics[gt_prefix]['FP'] += fp
        metrics[gt_prefix]['TN'] += tn
        metrics[gt_prefix]['FN'] += fn

        row['Predicted Prefix'] = gt_prefix
        row['Predicted Score'] = predicted_score
        row['TP'], row['FP'], row['TN'], row['FN'] = tp, fp, tn, fn

    save_predictions_and_metrics(df, metrics, csv_path)

# Set directories and CSV path
image_dirs = {
    'f': "D:/PATENT/random_test_images/Fibrosis",
    'i': "D:/PATENT/random_test_images/Inflammation",
    'b': "D:/PATENT/random_test_images/Ballooning",
    's': "D:/PATENT/random_test_images/Steatosis"
}
csv_path = "D:/PATENT/ground_truth_Scores_balanced.csv"

# Run the prediction and evaluation
predict_and_evaluate(csv_path, image_dirs)


In [ ]:
# new SMOTE bal

In [3]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import load_model

# Define focal_loss_fixed function again if it is not already available
def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        import tensorflow.keras.backend as K
        alpha_t = y_true * alpha + (K.ones_like(y_true) - y_true) * (1 - alpha)
        p_t = y_true * y_pred + (K.ones_like(y_true) - y_true) * (1 - y_pred)
        focal_loss = -alpha_t * K.pow((1 - p_t), gamma) * K.log(p_t + K.epsilon())
        return K.mean(focal_loss)
    return focal_loss_fixed

# Load the models for each disease (register custom loss function)
fibrosis_model = load_model("D:/PATENT/Model_testing/fibrosis_modified_model25.h5", custom_objects={'focal_loss_fixed': focal_loss()})
inflammation_model = load_model("D:/PATENT/Model_testing/inflam_balanced_modified25.h5", custom_objects={'focal_loss_fixed': focal_loss()})
ballooning_model = load_model("D:/PATENT/Model_testing/SMOTE_ballooning10.h5", custom_objects={'focal_loss_fixed': focal_loss()})
steatosis_model = load_model("D:/PATENT/Model_testing/steatosis_modified15.h5", custom_objects={'focal_loss_fixed': focal_loss()})

# Function to preprocess images for fibrosis
def preprocess_image_fibrosis(image_path):
    img = load_img(image_path, target_size=(299, 299))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    return img_array

# Function to predict image scores based on model type
def predict_image_scores(image_name, image_path):
    if image_name.startswith('f'):
        img_array = preprocess_image_fibrosis(image_path)
        model = fibrosis_model
    else:
        img = load_img(image_path, target_size=(299, 299))
        img_array = img_to_array(img) / 255.0
        img_array = np.expand_dims(img_array, axis=0)

        if image_name.startswith('i'):
            model = inflammation_model
        elif image_name.startswith('b'):
            model = ballooning_model
        elif image_name.startswith('s'):
            model = steatosis_model
        else:
            raise ValueError(f"Unknown disease prefix for {image_name}")

    predictions = model.predict(img_array)
    return np.argmax(predictions) if predictions.shape[-1] > 1 else int(predictions[0][0] > 0.5)

# Calculate metrics based on the new logic
def calculate_metrics(predicted_score, ground_truth_score):
    if ground_truth_score == predicted_score :
        return "TP", 1, 0, 0, 0
    elif ground_truth_score == 0 and predicted_score != 0:
        return "FP", 0, 1, 0, 0
    elif ground_truth_score != 0 and predicted_score == 0:
        return "FN", 0, 0, 0, 1
    elif ground_truth_score != predicted_score:
        return "TN", 0, 0, 1, 0
    else:
        return "", 0, 0, 0, 0

# Predict and save results along with individual metrics
def predict_and_save_results(csv_path, random_image_dirs):
    df = pd.read_csv(csv_path, dtype={'Ground Truth Prefix': str, 'Ground Truth Score': int})
    df['Predicted Prefix'] = ""
    df['Predicted Score'] = np.nan
    df['Performance'] = ""
    df['TP'], df['FP'], df['TN'], df['FN'] = 0, 0, 0, 0

    # Track metrics for each model
    metrics = {
        'f': {'TP': 0, 'FP': 0, 'TN': 0, 'FN': 0},
        'i': {'TP': 0, 'FP': 0, 'TN': 0, 'FN': 0},
        'b': {'TP': 0, 'FP': 0, 'TN': 0, 'FN': 0},
        's': {'TP': 0, 'FP': 0, 'TN': 0, 'FN': 0}
    }

    for idx, row in df.iterrows():
        image_name = row['Image Name']
        ground_truth_prefix = row['Ground Truth Prefix']
        ground_truth_score = row['Ground Truth Score']

        if ground_truth_prefix == 'f':
            image_dir = random_image_dirs["Fibrosis"]
        elif ground_truth_prefix == 'i':
            image_dir = random_image_dirs["Inflammation"]
        elif ground_truth_prefix == 'b':
            image_dir = random_image_dirs["Ballooning"]
        elif ground_truth_prefix == 's':
            image_dir = random_image_dirs["Steatosis"]
        else:
            continue

        image_path = os.path.join(image_dir, image_name + ".jpg")
        predicted_score = predict_image_scores(image_name, image_path)
        performance, tp, fp, tn, fn = calculate_metrics(predicted_score, ground_truth_score)

        df.at[idx, 'Predicted Prefix'] = ground_truth_prefix
        df.at[idx, 'Predicted Score'] = predicted_score
        df.at[idx, 'Performance'] = performance
        df.at[idx, 'TP'] = tp
        df.at[idx, 'FP'] = fp
        df.at[idx, 'TN'] = tn
        df.at[idx, 'FN'] = fn

        metrics[ground_truth_prefix]['TP'] += tp
        metrics[ground_truth_prefix]['FP'] += fp
        metrics[ground_truth_prefix]['TN'] += tn
        metrics[ground_truth_prefix]['FN'] += fn

    # Calculate and print metrics for each model
    for prefix, data in metrics.items():
        tp, fp, tn, fn = data['TP'], data['FP'], data['TN'], data['FN']
        total_predictions = tp + fp + tn + fn

        precision = tp / (tp + fp) if (tp + fp) != 0 else 0
        recall = tp / (tp + fn) if (tp + fn) != 0 else 0
        f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) != 0 else 0
        iou = tp / (tp + fp + fn) if (tp + fp + fn) != 0 else 0
        accuracy = (tp + tn) / total_predictions if total_predictions != 0 else 0

        print(f"\nMetrics for {prefix.upper()} model:")
        print(f"  Accuracy: {accuracy:.2f}")
        print(f"  Precision: {precision:.2f}")
        print(f"  Recall: {recall:.2f}")
        print(f"  F1-Score: {f1_score:.2f}")
        print(f"  IoU: {iou:.2f}")

    df.to_csv(csv_path, index=False)
    print("\nPredictions completed and saved with performance metrics in CSV.")

# Base path to random images for each disease
random_image_dirs = {
    "Fibrosis": "D:/PATENT/random_test_images/Fibrosis",
    "Inflammation": "D:/PATENT/random_test_images/Inflammation",
    "Ballooning": "D:/PATENT/random_test_images/Ballooning",
    "Steatosis": "D:/PATENT/random_test_images/Steatosis"
}

# Path to the ground truth CSV file
csv_path = "D:/PATENT/afterSMOTEbal.csv"

# Predict and save results for all diseases
predict_and_save_results(csv_path, random_image_dirs)



Metrics for F model:
  Accuracy: 0.97
  Precision: 1.00
  Recall: 0.97
  F1-Score: 0.98
  IoU: 0.97

Metrics for I model:
  Accuracy: 0.76
  Precision: 0.97
  Recall: 0.75
  F1-Score: 0.84
  IoU: 0.73

Metrics for B model:
  Accuracy: 0.90
  Precision: 1.00
  Recall: 0.90
  F1-Score: 0.95
  IoU: 0.90

Metrics for S model:
  Accuracy: 0.98
  Precision: 1.00
  Recall: 0.98
  F1-Score: 0.99
  IoU: 0.98

Predictions completed and saved with performance metrics in CSV.
